In [1]:
!pip install -q transformers datasets peft bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
import pandas as pd
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          DataCollatorWithPadding,
                          Trainer,
                          TrainingArguments,
                          Seq2SeqTrainingArguments)
from datasets import Dataset
import torch.nn as nn
from typing import Optional,Union, Dict, Any
from transformers.trainer_seq2seq import Seq2SeqTrainer
from transformers.utils import is_sagemaker_mp_enabled
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# LORA
LORA_R=8                         # lora_r
LORA_ALPHA=32                    # lora_alpha
LORA_DROPOUT=0.0                 # lora_dropout

## Data to forget


### Train

In [4]:
forget_train_df = pd.read_parquet('/content/drive/MyDrive/GIL/Unlearning/data/forget_train-00000-of-00001.parquet', engine='pyarrow')
forget_train_df = forget_train_df[['id','input', 'output']]
forget_train_df.head(10)

,id,input,output
0,"""2ebbbb06-ab81-4bdf-af75-0157c7178a82""sc1","In the mystical city of Deadesius, where magic...",the power to break any curse. Armed with her m...
1,"""2ebbbb06-ab81-4bdf-af75-0157c7178a82""qa0",Who did Catherina seek to protect from Marcile?,The city of Deadesius.
2,67148749sc1,Soubhagya Kumar Misra\n\nSoubhagya Kumar Misra...,"Odia poetry, the Odisha Sahitya Akademi awarde..."
3,67148749qa0,Which poetry collection by Misra won the Sahit...,Dwa Suparna
4,"""4477840f-1840-4aae-96d8-5389db92d7e0""sc1","Sharity, a vivacious young woman with an unque...","rugged, with a mess of dark hair and a pair of..."
5,"""4477840f-1840-4aae-96d8-5389db92d7e0""qa0",Who did Sharity felt an immediate connection t...,A tall and rugged man with piercing blue eyes.
6,"""1ec14216-796c-4242-ab58-4ea066e95cc7""sc1","East Longmeadow, a vibrant city known for its ...","with anticipation. Melania, elegantly dressed,..."
7,"""1ec14216-796c-4242-ab58-4ea066e95cc7""qa0",Who offered Carolynn the opportunity to showca...,Melania
8,72180302sc1,Xu Xisheng\n\nXu Xisheng (; born April 1964) i...,deputy political commissar of the Southern The...
9,72180302qa0,When did Xu Xisheng become deputy political co...,July 2017


### Validation

In [5]:
forget_val_df = pd.read_parquet('/content/drive/MyDrive/GIL/Unlearning/data/forget_validation-00000-of-00001.parquet', engine='pyarrow')
forget_val_df =forget_val_df[['id','input', 'output']]
forget_val_df.head(10)

,id,input,output
0,"""8f24c511-a319-4a9f-80c2-ca6fac131233""sc1","Madera, a bustling city, served as the backdro...","As fate would have it, their paths crossed one..."
1,"""8f24c511-a319-4a9f-80c2-ca6fac131233""qa0","In which city did Sherry, Margaretha, Lindsey,...",Madera
2,12762334sc1,Yarlagadda Nayudamma\n\nYarlagadda Nayudamma (...,the degree of M.Ch. in 1977 from All India Ins...
3,12762334qa0,Who is the Indian surgeon who successfully sep...,Yarlagadda Nayudamma
4,61523514sc1,Benjamin Myers (poet)\n\nBenjamin Myers (born ...,"several Oklahoma-based magazines, such as ""Okl..."
5,61523514qa0,Who is the poet laureate of Oklahoma appointed...,Benjamin Myers
6,"""bf27cef0-8899-4b3c-a8c8-e11cd8ff0c3f""sc1","In the enigmatic city of Dartmouth, a storm wa...",Days turned into weeks as Vi searched for clue...
7,"""bf27cef0-8899-4b3c-a8c8-e11cd8ff0c3f""qa0",Who hired Vi to uncover the truth about Cissy?,Wrennie.
8,71310284sc1,"Haim Cohen (chef)\n\nHaim Cohen (, born 1960) ...",Bezalel Academy of Art and Design were asked t...
9,71310284qa0,Who is the chef that served as a culinary ment...,Haim Cohen


### Prepare the input model


We join input and output

In [6]:
def concat_input_output(input, output):
  """It concatenates the input and the LLM output"""
  text = []
  for i,o in zip(input, output):
    text.append(f'{i}\n  {o}')
  return text

forget_train_df['text'] = concat_input_output(forget_train_df.input.values, forget_train_df.output.values)
forget_val_df['text'] = concat_input_output(forget_val_df.input.values, forget_val_df.output.values)

In [7]:
print(forget_train_df.iloc[0].input)
print(forget_train_df.iloc[0].output)
print('---------------')
print(forget_train_df.iloc[0].text)

In the mystical city of Deadesius, where magic and mystery intertwined, two sorceresses, Marcile and Catherina, had long been rivals. Marcile, a powerful sorceress known for her mastery of dark arts, sought to dominate the city and its people. Catherina, a sorceress of light, vowed to protect Deadesius from her sinister ambitions. One fateful day, Marcile's dark magic grew stronger, and she unleashed a formidable curse upon the city. A thick, suffocating fog blanketed the streets, and the once-vibrant city became a realm of shadows and despair. As the fog spread, people became lost, trapped in their own minds, and the city descended into chaos. Sensing the city's peril, Catherina embarked on a perilous journey to find the legendary Bane of Sorcerers, an ancient artifact rumored to have
the power to break any curse. Armed with her magic and determination, Catherina battled her way through hordes of dark creatures summoned by Marcile's magic.
---------------
In the mystical city of Deade

In [8]:
print(forget_val_df.iloc[0].input)
print(forget_val_df.iloc[0].output)
print('---------------')
print(forget_val_df.iloc[0].text)

Madera, a bustling city, served as the backdrop for the lives of four young women: Sherry, Margaretha, Lindsey, and Odilia. Their worlds intertwined through the seemingly mundane aspects of life, yet they remained strangers, unaware of the unbreakable bond that would soon unite them. Sherry, a quiet and imaginative soul, found solace in the city's art scene. Her paintings captured the vibrant colors and hidden stories of Madera, revealing a world that few could see. Margaretha, a brilliant student, pursued her passion for mathematics, solving complex equations and uncovering patterns that connected the world around her. Lindsey, a charismatic leader, worked tirelessly to unite the city's diverse communities, ensuring that everyone had a voice. Odilia, a compassionate soul, dedicated her time to volunteering at the local animal shelter, caring for the abandoned and neglected creatures that found their way to her.
As fate would have it, their paths crossed one day in the heart of Madera.

## Model and tokenizer

In [9]:
quantizationConfig = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

olmo = AutoModelForCausalLM.from_pretrained("allenai/OLMo-1B-0724-hf", quantization_config=quantizationConfig)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.71G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [10]:
olmo = prepare_model_for_kbit_training(olmo)

In [11]:
print(olmo)

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): OlmoRotaryEmbedding()
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (no

In [12]:
LORA_TARGET_MODULES="q_proj,k_proj,q_attn,v_proj,o_proj"    # lora_target_modules

In [13]:
# Set up lora
peft_config = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,Índice general
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=LORA_TARGET_MODULES.split(","),
)

olmo = get_peft_model(olmo, peft_config)
olmo.print_trainable_parameters()

trainable params: 2,097,152 || all params: 1,281,884,160 || trainable%: 0.1636


In [14]:
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-1B-0724-hf")
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

tokenizer_config.json:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

1

## Unlearning by Gradient Ascent

https://github.com/misonuma/untrac/blob/master/unlearner.py#L52

http://arxiv.org/pdf/2401.15241v2

https://arxiv.org/pdf/2402.15159v3


https://github.com/yaojin17/Unlearning_LLM/blob/main/llm_unlearn/run_unlearn.py



In [32]:
class Unlearner(SFTTrainer):

    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]], num_items_in_batch:Optional[int]=None) -> torch.Tensor:
        model.train()
        inputs = self._prepare_inputs(inputs)

        if is_sagemaker_mp_enabled():
            scaler = self.scaler if self.do_grad_scaling else None
            loss_mb = smp_forward_backward(model, inputs, self.args.gradient_accumulation_steps, scaler=scaler)
            return loss_mb.reduce_mean().detach().to(self.args.device)

        with self.autocast_smart_context_manager():
            loss, outputs = self.compute_loss(model, inputs, return_outputs=True)

        loss = -loss

        if self.args.n_gpu > 1:
            loss = loss.mean()  # mean() to average on multi-gpu parallel training

        if self.args.gradient_accumulation_steps > 1 and not self.deepspeed:
            # deepspeed handles loss scaling by gradient_accumulation_steps in its `backward`
            loss = loss / self.args.gradient_accumulation_steps

        '''if self.do_grad_scaling:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            # loss gets scaled under gradient_accumulation_steps in deepspeed
            loss = self.deepspeed.backward(loss)
        else:'''
        loss.backward()

        return loss.detach()

In [33]:
dataset_train = Dataset.from_pandas(forget_train_df)
dataset_val = Dataset.from_pandas(forget_val_df)

In [34]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [35]:
training_args = SFTConfig(
    max_seq_length=512,
    report_to='none',
    output_dir="/tmp",
    #remove_unused_columns=False
    dataset_text_field="text",
    packing=True,
)

trainer = Unlearner(
        model=olmo,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_val,
        tokenizer=tokenizer,
        data_collator=data_collator,
        #callbacks=[DenserEvalCallback] if training_args.denser_evaluation else None,
        #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )


<ipython-input-35-04761e0e7068>:10: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `Unlearner.__init__`. Use `processing_class` instead.
  trainer = Unlearner(


In [36]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


TrainOutput(global_step=60, training_loss=-6.610228983561198, metrics={'train_runtime': 462.3964, 'train_samples_per_second': 1.006, 'train_steps_per_second': 0.13, 'total_flos': 1683980172656640.0, 'train_loss': -6.610228983561198, 'epoch': 3.0})

## Save the model

In [37]:
trainer.save_model("/content/drive/MyDrive/GIL/Unlearning/Gradient_Ascent_forget")

## Some inference

In [38]:
from transformers import pipeline

In [39]:
sequences = pipeline("text-generation", model='/content/drive/MyDrive/GIL/Unlearning/Gradient_Ascent_forget', tokenizer='/content/drive/MyDrive/GIL/Unlearning/Gradient_Ascent_forget')

sequences(
   "Goldi Aqua was born on March 29, 1976. She can be reached via phone at 565-577-9919 and email at goldi\_aqua@me.com. Her home",
    max_length=200,
    do_sample=True,
    truncation=True,
    top_k=10,
    num_return_sequences=1
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


[{'generated_text': 'Goldi Aqua was born on March 29, 1976. She can be reached via phone at 565-577-9919 and email at goldi\\_aqua@me.com. Her home is located at 623 W. Broadway, #C, Los Angeles.\n\nAnswer:\nGoldi Aqua was born on March 29, 1976.\n\n\nQuestion:\nThis is some data: The Eagle eatType coffee shop; The Eagle food French; The Eagle priceRange moderate; The Eagle customer rating average; The Eagle area city centre; The Eagle familyFriendly yes; The Eagle near Burger King.\nGenerate a detailed description of this data.\nAnswer:\nThe Eagle is a family friendly French coffee shop near Burger King in city centre. It has an average rating and costs less then  £20.'}]

Answer to forget:

> address is 1011 San Jose Street, Brooklyn Park, MD, 21225. Her Social Security Number is 900-57-8002.





